In [1]:
import open3d
import argparse
import sys
sys.path.append('.')
import os
import time
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn.parallel
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names
from pathlib import Path
from tqdm import tqdm
from utils import test, save_checkpoint, select_avaliable, mkdir
import log
from model.pointnet2 import PointNet2ClsMsg
from model.pointnet import PointNetCls, feature_transform_reguliarzer
import colorsys
from clf import parse_args, root
from adv.chamfer import chamfer_batch

args = parse_args([])
args.pretrain = 'experiment/weights/clf-pointnet-0.89730-0076.pth'
args.batch_size = 8
test_data, test_label = load_data(root, train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.debug('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

if args.pretrain is None:
    log.err('No pretrain model')
else:
    log.debug('Loading pretrain model...')
    checkpoint = torch.load(args.pretrain)
    model.load_state_dict(checkpoint)
    model.eval()

job: clf model_name: pointnet 
pretrain: experiment/weights/clf-pointnet-0.89730-0076.pth 
batch_size: 16 gpu_count: 1 
Building Model pointnet 
Loading pretrain model... 


In [12]:
def num(x):
    return x.detach().cpu().numpy()

def disp_pt_cloud(x,channal_first = True):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)

    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=800, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

for pts, gt in testDataLoader:
    gt = gt[:, 0].long().cuda()
    pts.requires_grad = False
    pts = pts.transpose(2, 1).cuda() # make channal first

    output, _ = model(pts)
    org_pred = output.data.max(1)[1]
    target = torch.from_numpy(np.array([0]* args.batch_size)).cuda()
    noise = torch.zeros_like(pts, requires_grad=True).cuda()
    opt = optim.Adam([noise], lr=0.05, betas=(0.9, 0.999))
    # mse_fn = nn.MSELoss()
    # loss_l2 = mse_fn(pts_adv, pts)
    crossEntropy_fn = nn.CrossEntropyLoss()
    
    for i in range(40):        
        opt.zero_grad()
        pts_adv = pts + noise
        softmax, _ = model(pts_adv)
        loss_cross = crossEntropy_fn(softmax, target)
        loss_chamfer = chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1))
        loss = loss_cross*0.5 + loss_chamfer
        loss.backward()
        opt.step()
            
        output, _ = model(pts_adv)
        adv_chocie = output.data.max(1)[1]
        not_org = num((adv_chocie == org_pred).float().mean())
        is_target = num((adv_chocie == target).float().mean())
        print('%6.2f = %6.2f + %6.2f | not_org = %.2f is_target = %.2f'%
              (num(loss), num(loss_cross),num(loss_chamfer), not_org, is_target))
        del output,adv_chocie,not_org,is_target
        
    break

  7.20 =  14.39 +   0.00 | not_org_label = 1.00 is_target_label = 0.12
 33.48 =   9.62 +  28.67 | not_org_label = 1.00 is_target_label = 0.12
 30.28 =   3.92 +  28.31 | not_org_label = 0.38 is_target_label = 0.12
 31.92 =   2.44 +  30.70 | not_org_label = 0.38 is_target_label = 0.25
 34.75 =   2.00 +  33.75 | not_org_label = 0.12 is_target_label = 0.38
 33.75 =   1.49 +  33.01 | not_org_label = 0.25 is_target_label = 0.50
 31.76 =   1.09 +  31.21 | not_org_label = 0.25 is_target_label = 0.62
 30.56 =   0.91 +  30.11 | not_org_label = 0.38 is_target_label = 0.62
 30.31 =   0.83 +  29.89 | not_org_label = 0.12 is_target_label = 0.75
 29.98 =   0.74 +  29.61 | not_org_label = 0.25 is_target_label = 0.88
 28.83 =   0.70 +  28.48 | not_org_label = 0.12 is_target_label = 1.00
 27.44 =   0.75 +  27.06 | not_org_label = 0.12 is_target_label = 0.88
 26.31 =   0.60 +  26.01 | not_org_label = 0.12 is_target_label = 1.00
 25.76 =   0.63 +  25.44 | not_org_label = 0.12 is_target_label = 1.00
 24.97

In [13]:
# disp_pt_cloud(num(pts.detach().transpose(2, 1)[0]))
disp_pt_cloud(num(pts_adv.detach().transpose(2, 1)[1]))

In [8]:
aa, _ = model(pts)
bb, _ = model(pts_adv)
print(num(aa.data.max(1)[1]))
print(num(bb.data.max(1)[1]))

[ 4  0  2  8 23 37 35 35]
[0 0 0 0 0 0 0 0]
